In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing
from sklearn.preprocessing import KBinsDiscretizer
import time, math, random
from typing import Tuple
import gym

In [3]:
env = gym.make('CartPole-v1')

In [4]:
?env.env

In [5]:
n_bins = ( 6 , 12 )
l=[ env.observation_space.low[2], -math.radians(50) ]
u=[ env.observation_space.high[2], math.radians(50) ]
def conv( _ , __ , angle, pole_velocity ) -> Tuple[int,...]:
    res= KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
    res.fit([l,u])
    return tuple(map(int,res.transform([[angle, pole_velocity]])[0]))

In [6]:
m= np.zeros(n_bins + (env.action_space.n,))
m.shape

(6, 12, 2)

In [8]:
def policy( state : tuple ):
    return np.argmax(m[state])

In [9]:
def new( reward : float ,  new_state : tuple , discount_factor=1 ) -> float:
    future_optimal_value = np.max(m[new_state])
    new_value = reward + discount_factor * future_optimal_value
    return new_value

In [10]:
def learning_rate(n : int , min_rate=0.01 ) -> float  :
    return max(min_rate, min(1.0, 1.0 - math.log10((n + 1) / 25)))

In [11]:
def exploration_rate(n : int, min_rate= 0.1 ) -> float :
    return max(min_rate, min(1, 1.0 - math.log10((n  + 1) / 25)))

In [ ]:
n_episodes = 1000 
for e in range(n_episodes):
    current_state, done = conv(*env.reset()), False
    while done==False:
        action = policy(current_state)
        if np.random.random() < exploration_rate(e) : 
            action = env.action_space.sample()
        obs, reward, done, _ = env.step(action)
        new_state = conv(*obs)
        lr = learning_rate(e)
        value = new(reward , new_state )
        old= m[current_state][action]
        m[current_state][action] = (1-lr)*old + lr*value
        current_state = new_state
        env.render()